# Smartsheet API Tutorial #2 | Excel file to Smartsheet

This is a tutorial for taking an excel sheet and uploading it to smartsheet.

resources:
1. [Smartsheet API DOCS](https://smartsheet-platform.github.io/api-docs/)
2. [Pandas Docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame)
4. [DCT Smartsheet Workspace](https://app.smartsheet.com/workspaces/8hfcJw2pVvc2MFf3vP34QVVJjv3r9jVXxCrWWhf1)
3. REVIEW THE #dct_learn-python channel for previous tutorials and .ipynb (jupyter notebooks)

### Dependencies

1. `pip install smartsheet-python-sdk`
2. `pip install pandas`
3. `pip install openpyxl`
4. `pip install cryptography`

### Imports

In [ ]:
import smartsheet
import pandas as pd 
import openpyxl
from cryptography.fernet import Fernet
import os

#### Step 0.5 | **VARIABLES**

adding this section so that you can assign all dynamic variables at the beginning of the script

In [ ]:
#the file path to your excel file, MUST MODIFY!
excel_file_path = r"C:\PATH\TO\EXCEL\SHEET.xlsx"

#store the folder_id for the SA-QTO Folder on smartsheet
saqto_folder_id = '7361149952911236'

#name of smartsheet you would like to create
new_sheet_name = 'CHOOSE_A_NAME_AND_PLACE_IT_HERE'

#### Step 1 | **Load Excel Sheet into Pandas**

In [ ]:
#load excel into pandas DataFrame
source_sheet = pd.read_excel(excel_file_path,engine='openpyxl')
source_sheet

#### Step 2 | **Log into Smartsheet**

In [ ]:
#grab token from environmental variables SEE TUTOTIRAL #1
token = bytes(os.environ.get("SMARTSHEET_ACCESS_TOKEN"), "utf-8")
key = os.environ.get("f")
f = Fernet(key)
stoken = f.decrypt(token).decode("utf-8")


#log into smartsheet
smart = smartsheet.Smartsheet(access_token=stoken)
smart.errors_as_exceptions(True)

#### Step 3 | **Create a Sheet in a folder**

In [ ]:
#this function is designed to create the data structure needed to for column names from excel onto smartsheet. It takes the first column in the sheet and makes it the primary column
def smartsheet_column(list):
    column_dict_list = []
    for column_name, i in zip(list, range(len(list))):
        column_dict = {}
        if i == 0:
            column_dict['primary'] = True
        column_dict['title'] = column_name
        column_dict['type'] = 'TEXT_NUMBER'
        column_dict_list.append(column_dict)
    return column_dict_list


In [ ]:
#grab all the names of the columns from the excel sheet
columns = smartsheet_column(source_sheet.columns)

In [ ]:
#Create the sheet specifications
sheet_spec = smartsheet.models.Sheet({
  'name': new_sheet_name,
  'columns': columns
})



#create the new folder and store the information about the new sheet in a variable called 'new_sheet'
response = smart.Folders.create_sheet_in_folder(
  saqto_folder_id,       # folder_id
  sheet_spec)
new_sheet = response.result

#### Step 3 | **Add rows to sheet**

In [ ]:
#take the sheet_id just created and assign it to a variable called sheet_id
sheet_id = new_sheet.to_dict().get('id')

In [ ]:
#create a list of column_ids
column_id_list = []
for i in new_sheet.to_dict().get('columns'):
    column_id_list.append(i.get('id'))

In [ ]:
#this code converts the pandas dataframe information to a list of lists, with each list have information from each row.
rows = source_sheet.values.tolist()

In [ ]:
#take the row information, column ids and sheet id and upload them to smartsheet.
for row in rows:
    new_row = smart.models.Row()
    new_row.to_bottom = True
    for item, column_id in zip(row, column_id_list):
        new_row.cells.append({
            'column_id': column_id,
            'value':item,
            'strict': False
        })
    response = smart.Sheets.add_rows(sheet_id, [new_row])